# Day 3

Full text of the task can be found on [AoC website](https://adventofcode.com/2023/day/3).

We arrived at the gondola lift that will take us up to the water source. But the gondolas don't work - they are missing a part! 

Another ChatGPT 4 & DALL·E illustration:

<img src="./ChatGPT_illustrations/day03.png" width="400" />

## Part One

We need to sum up the part numbers of all engine parts. The parts are marked by an adjacent symbol. 

This sounds tricky. Given a grid schematic, we need to figure out which of the numbers are adjacent to a symbol (in all directions). The dot (`.`) is not a symbol. Those number will be used for the sum.

|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  |
|---|---|---|---|---|---|---|---|---|---|----|
| 0 | 4 | 6 | 7 | . | . | **1** | **1** | **4** | . | .  |
| 1 | . | . | . | * | . | . | . | . | . | .  |
| 2 | . | . | 3 | 5 | . | . | 6 | 3 | 3 | .  |
| 3 | . | . | . | . | . | . | # | . | . | .  |
| 4 | 6 | 1 | 7 | * | . | . | . | . | . | .  |
| 5 | . | . | . | . | . | + | . | **5** | **8** | .  |
| 6 | . | . | 5 | 9 | 2 | . | . | . | . | .  |
| 7 | . | . | . | . | . | 7 | 5 | 5 | . | .  |
| 8 | . | . | . | $ | . | * | . | . | . | .  |
| 9 | . | 6 | 6 | 4 | . | 5 | 9 | 8 | . | .  |


In this example only `114` in the zero line and `58` in the 5th line are not adjacent to a symbol. The sum of all other numbers is equal to `4361`.


Looks like I need to get the position of each number and then check the position around it. 

In [1]:
example_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

def process_input(input, example = False):

    symbol_dict = dict()
    numbers_dict = dict()
    y = 0
    if not example:
        input = open(input).read()

    for line in input.splitlines():
        num = ""
        for x, c in enumerate(line):
            # if c in a number add to num
            if c in '0123456789':
                num += c
            else:
                # if num is not empty add to numbers_dict
                if num != "":
                    numbers_dict[(x-1, y)] = int(num)
                    num = ""
                # if c is not a dot add to symbol_dict
                if c != '.':
                    # I don't really need symbol_dict, a set would be enough
                    # but in case I might need it later, I'll keep it
                    symbol_dict[(x, y)] = c
        if num != "":
            numbers_dict[(x, y)] = int(num)
        y += 1

    return symbol_dict, numbers_dict

process_input(example_input, True)

({(3, 1): '*',
  (6, 3): '#',
  (3, 4): '*',
  (5, 5): '+',
  (3, 8): '$',
  (5, 8): '*'},
 {(2, 0): 467,
  (7, 0): 114,
  (3, 2): 35,
  (8, 2): 633,
  (2, 4): 617,
  (8, 5): 58,
  (4, 6): 592,
  (8, 7): 755,
  (3, 9): 664,
  (7, 9): 598})

In [2]:
def scan_grid(symbol_dict, numbers_dict):
    parts_sum = 0
    # for each number in numbers_dict, check if in x+-1 and y+-1 there is a symbol
    for (x, y), num in numbers_dict.items():
        # get the number of digits of num
        num_digits = len(str(num))
        # check if there is a symbol in (x_min-1 to x_max+1) and y+-1
        x_min = x - num_digits
        x_max = x + 1
        # create all possible combinations of x1, y1 
        # where x is within x_min and x_max and y is y-1 or y+1
        xs = range(x_min, x_max+1)
        ys = [y-1, y+1]
        # take only the xs and ys >= 0
        xs = [x1 for x1 in xs if x1 >= 0]
        ys = [y1 for y1 in ys if y1 >= 0]
        pos = [(x1, y1) for x1 in xs for y1 in ys]
        pos += [(x1, y) for x1 in [x_min, x_max] if x1 >= 0]
        if any([symbol_dict.get(p) for p in pos]):
            parts_sum += num

    return parts_sum

scan_grid(*process_input(example_input, True))

4361

I needed debug for a bit bc I forgot to add numbers at the end of the line in  `process_input`. 

In [3]:
def print_matrix(input_string):
    # Split the input string into rows
    rows = input_string.split("\n")
    # get the max length of the rows
    max_len = max([len(row) for row in rows])
    
    # Create a matrix of the max length
    matrix = [[' ' for i in range(max_len)] for j in range(len(rows))]

    # Populate the matrix with characters from the input string
    for i, row in enumerate(rows):
        for j, char in enumerate(row):
            matrix[i][j] = char
    # Print the matrix with row and column numbers
    print(f"   | {' | '.join([str(i) if i < 10 else ' ' for i in range(max_len)])} |")
    print(f" {'-'*(max_len*4 + 4)}")
    for i, row in enumerate(matrix):
        if i < 10:
            print(f" {i} | {' | '.join(row)} |")
        else:
            print(f"{i} | {' | '.join(row)} |")

# Input string
input_string = """....
...1
989*
._--"""

# Print the matrix
print_matrix(input_string)


   | 0 | 1 | 2 | 3 |
 --------------------
 0 | . | . | . | . |
 1 | . | . | . | 1 |
 2 | 9 | 8 | 9 | * |
 3 | . | _ | - | - |


In [4]:
corner_case = """111$12
+.....
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 123)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | 1 | 1 | 1 | $ | 1 | 2 |
 1 | + | . | . | . | . | . |
 2 | . | . | 1 | 3 | . | % |


In [5]:
corner_case = """111$12
+...8.
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 131)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | 1 | 1 | 1 | $ | 1 | 2 |
 1 | + | . | . | . | 8 | . |
 2 | . | . | 1 | 3 | . | % |


In [6]:
corner_case = """..$$..
+...8.
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 8)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | . | . | $ | $ | . | . |
 1 | + | . | . | . | 8 | . |
 2 | . | . | 1 | 3 | . | % |


In [7]:
corner_case = """1.3
.5.
7.9"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
 0 | 1 | . | 3 |
 1 | . | 5 | . |
 2 | 7 | . | 9 |


In [8]:
corner_case = """1*3
*5*
7*9"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 25)

   | 0 | 1 | 2 |
 ----------------
 0 | 1 | * | 3 |
 1 | * | 5 | * |
 2 | 7 | * | 9 |


In [9]:
corner_case = """.$12"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 12)

   | 0 | 1 | 2 | 3 |
 --------------------
 0 | . | $ | 1 | 2 |


In [10]:
corner_case = """***"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
 0 | * | * | * |


In [11]:
corner_case_from_reddit = """333.3
...*."""
print_matrix(corner_case_from_reddit)
assert(scan_grid(*process_input(corner_case_from_reddit, True)) == 336)

   | 0 | 1 | 2 | 3 | 4 |
 ------------------------
 0 | 3 | 3 | 3 | . | 3 |
 1 | . | . | . | * | . |


In [12]:
corner_case = """1.3
.5*
..9"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 17)

   | 0 | 1 | 2 |
 ----------------
 0 | 1 | . | 3 |
 1 | . | 5 | * |
 2 | . | . | 9 |


In [13]:
corner_case = """*.*
123"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 123)

   | 0 | 1 | 2 |
 ----------------
 0 | * | . | * |
 1 | 1 | 2 | 3 |


In [14]:
scan_grid(*process_input('./inputs/day03.txt'))

539637

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.

## Part Two

In this part, we are actually only interested in '*' near two numbers. Then we calculate the gear ratio by multiplying those numbers and adding them to the sum.

|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  |
|---|---|---|---|---|---|---|---|---|---|----|
| 0 | **4** | **6** | **7** | . | . | 1 | 1 | 4 | . | .  |
| 1 | . | . | . | * | . | . | . | . | . | .  |
| 2 | . | . | **3** | **5** | . | . | 6 | 3 | 3 | .  |
| 3 | . | . | . | . | . | . | # | . | . | .  |
| 4 | 6 | 1 | 7 | * | . | . | . | . | . | .  |
| 5 | . | . | . | . | . | + | . | 5 | 8 | .  |
| 6 | . | . | 5 | 9 | 2 | . | . | . | . | .  |
| 7 | . | . | . | . | . | **7** | **5** | **5** | . | .  |
| 8 | . | . | . | $ | . | * | . | . | . | .  |
| 9 | . | 6 | 6 | 4 | . | **5** | **9** | **8** | . | .  |

There are two gears here - `467` and `35` with gear ratio `16345` and `755` and `598` with gear ratio `451490`. Together, the sum is `467835`.

In [15]:
def process_input_for_gears(input, example = False):

    gears = set()
    numbers_idx = dict()
    numbers_dict = dict()
    y = 0
    idx = 0
    if not example:
        input = open(input).read()

    for line in input.splitlines():
        num = ""
        for x, c in enumerate(line):
            # if c in a number add to num
            if c in '0123456789':
                num += c
            else:
                # if num is not empty add to numbers_dict
                if num != "":
                    numbers_idx[idx] = num
                    # for each position of the number add the idx
                    for i in range(len(num)):
                        numbers_dict[(x-i-1, y)] = idx
                    num = ""
                    idx += 1
                # if c is not a dot add to symbol_dict
                if c == '*':
                    gears.add((x, y))
        if num != "":
            numbers_idx[idx] = num
            # for each position of the number add the idx
            for i in range(len(num)):
                numbers_dict[(x-i, y)] = idx
            num = ""
            idx += 1
        y += 1

    return gears, numbers_idx, numbers_dict


In [16]:
def scan_grid_for_gears(gears, numbers_idx, numbers_dict):
    gears_sum = 0
    # for each '*' in symbol_dict, check if in x+-1 and y+-1 there are exactly two numbers
    for (x, y) in gears:
        # check if there are exactly two numbers in (x-1 to x+1) and (y-1 to y+1)
        xs = [x-1, x, x+1]
        ys = [y-1, y, y+1]
        # take only the xs and ys >= 0
        xs = [x1 for x1 in xs if x1 >= 0]
        ys = [y1 for y1 in ys if y1 >= 0]
        pos = [(x1, y1) for x1 in xs for y1 in ys if (x1, y1) != (x, y)]
        # get the idx of the numbers in the positions
        idxs = [numbers_dict.get(p) for p in pos if numbers_dict.get(p) is not None]
        idxs = list(set(idxs))
        if len(idxs) == 2:
            # get the numbers
            nums = [numbers_idx.get(i) for i in idxs]
            # multiply the numbers
            gears_sum += int(nums[0]) * int(nums[1])

    return gears_sum

scan_grid_for_gears(*process_input_for_gears(example_input, True))

467835

In [17]:
corner_case = """1.3
.5*
..9"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
 0 | 1 | . | 3 |
 1 | . | 5 | * |
 2 | . | . | 9 |


In [18]:
corner_case = """*.*
123"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
 0 | * | . | * |
 1 | 1 | 2 | 3 |


In [19]:
corner_case = """.*.
1.3"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 3)

   | 0 | 1 | 2 |
 ----------------
 0 | . | * | . |
 1 | 1 | . | 3 |


In [20]:
corner_case = """***"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
 0 | * | * | * |


In [21]:
corner_case = """111*12
*...8.
..13.*"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | 1 | 1 | 1 | * | 1 | 2 |
 1 | * | . | . | . | 8 | . |
 2 | . | . | 1 | 3 | . | * |


In [22]:
corner_case = """111*12
*...8.
..13**"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 8*13)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | 1 | 1 | 1 | * | 1 | 2 |
 1 | * | . | . | . | 8 | . |
 2 | . | . | 1 | 3 | * | * |


In [23]:
corner_case = """111*12
*.*.8.
..13**"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 8*13+111*13)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
 0 | 1 | 1 | 1 | * | 1 | 2 |
 1 | * | . | * | . | 8 | . |
 2 | . | . | 1 | 3 | * | * |


In [24]:
scan_grid_for_gears(*process_input_for_gears('./inputs/day03.txt'))

82818007

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.